In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras.layers import Input, Embedding, LSTM, Dense, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import sequence, text
import gensim
import re, string
import tinysegmenter
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import pickle
import feather

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

                          tqdm:   1.0KiB
                         Model:   1.0KiB
                    Sequential:   1.0KiB
                     Embedding:   1.0KiB
                          LSTM:   1.0KiB
                         Dense:   1.0KiB
                       Flatten:   1.0KiB
                   Concatenate:   1.0KiB
                       Dropout:   1.0KiB
              SpatialDropout1D:   1.0KiB


In [2]:
data = pd.read_csv("Merged/JPN-1/data_joined_deu.csv", encoding='utf-8')
data.head()

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20,24,25,27,34,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,date,companyName,currency,language,countryName,distributorName,ageGroup,articleId,colors,...,CURRENT_PRICE_IN_EUR_OUTPUT,INITIAL_PRICE_IN_SELECTED_CURRENCY_OUTPUT,CURRENT_PRICE_IN_SELECTED_CURRENCY_OUTPUT,SELECTED_CURRENCY_OUTPUT,PRODUCT_INTRODUCTION_DATE_OUTPUT,DISCOUNTED_SINCE_OUTPUT,PRODUCT_EXIT_DATE_OUTPUT,PRODUCT_DESCRIPTION_OUTPUT,PRODUCT_URL_OUTPUT,IMAGE_SERVER_URL_OUTPUT
0,0,03/17/2016,adidas,JPY,ja-JP,jpn,own ecom,adults,AA0647,ホワイト,...,24.3,24.3,24.3,EUR,2/29/2016,9/18/2017,11/21/2017,3æ¬ç·ã®ãã¶ã¤ã³ãæ°ãããªã£ãã¬...,https://shop.adidas.jp/products/AA0647/,http://usporamap287.am.adsint.biz/zoomimages/1...
1,1,03/22/2016,adidas,JPY,ja-JP,jpn,own ecom,adults,AA0647,ホワイト,...,24.3,24.3,24.3,EUR,2/29/2016,9/18/2017,11/21/2017,3æ¬ç·ã®ãã¶ã¤ã³ãæ°ãããªã£ãã¬...,https://shop.adidas.jp/products/AA0647/,http://usporamap287.am.adsint.biz/zoomimages/1...
2,2,03/30/2016,adidas,JPY,ja-JP,jpn,own ecom,adults,AA0647,ホワイト,...,24.3,24.3,24.3,EUR,2/29/2016,9/18/2017,11/21/2017,3æ¬ç·ã®ãã¶ã¤ã³ãæ°ãããªã£ãã¬...,https://shop.adidas.jp/products/AA0647/,http://usporamap287.am.adsint.biz/zoomimages/1...
3,3,04/06/2016,adidas,JPY,ja-JP,jpn,own ecom,adults,AA0647,ホワイト,...,24.3,24.3,24.3,EUR,2/29/2016,9/18/2017,11/21/2017,3æ¬ç·ã®ãã¶ã¤ã³ãæ°ãããªã£ãã¬...,https://shop.adidas.jp/products/AA0647/,http://usporamap287.am.adsint.biz/zoomimages/1...
4,4,04/14/2016,adidas,JPY,ja-JP,jpn,own ecom,adults,AA0647,ホワイト,...,24.3,24.3,24.3,EUR,2/29/2016,9/18/2017,11/21/2017,3æ¬ç·ã®ãã¶ã¤ã³ãæ°ãããªã£ãã¬...,https://shop.adidas.jp/products/AA0647/,http://usporamap287.am.adsint.biz/zoomimages/1...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008008 entries, 0 to 2008007
Data columns (total 62 columns):
Unnamed: 0                                   int64
date                                         object
companyName                                  object
currency                                     object
language                                     object
countryName                                  object
distributorName                              object
ageGroup                                     object
articleId                                    object
colors                                       object
colorGroup                                   object
consumerRating                               float64
currentPrice                                 float64
description                                  object
discountedSince                              object
division                                     object
gender                                       object
gr

In [3]:
data.date = pd.to_datetime(data.date, infer_datetime_format=True)

In [4]:
data = data[data.date.dt.year > 2016]

In [5]:
max(data.date)

Timestamp('2018-10-15 00:00:00')

In [6]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [7]:
data.columns

Index(['Unnamed: 0', 'date', 'companyName', 'currency', 'language',
       'countryName', 'distributorName', 'ageGroup', 'articleId', 'colors',
       'colorGroup', 'consumerRating', 'currentPrice', 'description',
       'discountedSince', 'division', 'gender', 'group', 'imageUrl',
       'initialPrice', 'modelNumber', 'name4', 'ped', 'pid', 'sportsCategory',
       'subBrand', 'technologies', 'type', 'url', 'key', 'COMPANY',
       'COUNTRY_OUTPUT', 'DISTRIBUTOR_OUTPUT', 'ARTICLE_ID_OUTPUT',
       'MODEL_NUMBER_OUTPUT', 'ARTICLE_NAME_OUTPUT', 'SUBBRAND_OUTPUT',
       'SPORTS_CATEGORY_OUTPUT', 'PRODUCT_DIVISION_OUTPUT',
       'PRODUCT_GROUP_OUTPUT', 'PRODUCT_TYPE_OUTPUT', 'FRANCHISE_OUTPUT',
       'TECHNOLOGIES_OUTPUT', 'COLOUR_GROUP_OUTPUT', 'COLOUR_OUTPUT',
       'GENDER_OUTPUT', 'AGE_GROUP_OUTPUT', 'CONSUMER_RATING_OUTPUT',
       'INITIAL_PRICE_IN_LOCAL_CURRENCY_OUTPUT',
       'CURRENT_PRICE_IN_LOCAL_CURRENCY_OUTPUT', 'LOCAL_CURRENCY_OUTPUT',
       'INITIAL_PRICE_IN_EUR_OUTP

In [8]:
data_req = data.loc[:,["date", "companyName", "distributorName", "ageGroup","description", "division", "gender", "group", "name4", "sportsCategory","subBrand", "SUBBRAND_OUTPUT"]]

In [10]:
data_req["ID"] = data_req.index

In [11]:
# data_req = data_req.drop_duplicates()

In [12]:
# data_req["description"]  = data_req["COMPANY"] +" " + data_req["ARTICLE NAME"] +" " + data_req["PRODUCT DESCRIPTION"]

In [13]:
# data_req = data_req.drop(["COUNTRY", "ARTICLE NAME", "PRODUCT DESCRIPTION", "PRODUCT URL"], axis = 1)
# data_req.columns = ["COMPANY", "subBrand", "description"]

In [14]:
data_req[["companyName", "distributorName", "ageGroup","description", "division", "gender", "group","name4", "sportsCategory", "subBrand"]] = data_req[["companyName", "distributorName", "ageGroup","description", "division", "gender", "group", "name4", "sportsCategory", "subBrand"]].fillna("unknown")

In [15]:
data_req = data_req.apply(lambda x: x.astype(str).str.lower())

In [16]:
data_req.isnull().sum()

date               0
companyName        0
distributorName    0
ageGroup           0
description        0
division           0
gender             0
group              0
name4              0
sportsCategory     0
subBrand           0
SUBBRAND_OUTPUT    0
ID                 0
dtype: int64

In [9]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data_req["SUBBRAND_OUTPUT"])

In [11]:
filename = "y_Jpn_lbl_enc.pkl"
filehandler = open(filename, 'wb')
pickle.dump(lbl_enc, filehandler)

In [18]:
data_req[["companyName", "distributorName", "ageGroup", "division", "gender", "group", "sportsCategory", "subBrand"]] = \
    data_req[["companyName", "distributorName", "ageGroup", "division", "gender", "group", "sportsCategory", "subBrand"]].apply(preprocessing.LabelEncoder().fit_transform)

In [19]:
# data_req["SUBRAND_OUTPUT_ENC"] = data_req[["SUBBRAND_OUTPUT"]].apply(preprocessing.LabelEncoder().fit_transform)

In [20]:
X = ["companyName", "distributorName", "ageGroup","description", "division", "gender", "group","name4", "sportsCategory", "subBrand"]

In [21]:
data_req["SUBBRAND_OUTPUT"].value_counts()

performance       870429
sport inspired    648380
Name: SUBBRAND_OUTPUT, dtype: int64

In [22]:
data_req["SUBRAND_OUTPUT_ENC"] = y

In [23]:
y = "SUBRAND_OUTPUT_ENC"

In [24]:
data_req.head()

,date,companyName,distributorName,ageGroup,description,division,gender,group,name4,sportsCategory,subBrand,SUBBRAND_OUTPUT,ID,SUBRAND_OUTPUT_ENC
489199,2017-01-02,0,0,0,幅の広めのラージサイズ。長時間の着用でも疲れを感じさせない人間工学から考えられたテンプル形状...,1,1,26,サングラス a164 6050 adivista l,12,5,performance,489199,0
489200,2017-01-11,0,0,0,幅の広めのラージサイズ。長時間の着用でも疲れを感じさせない人間工学から考えられたテンプル形状...,1,1,26,サングラス a164 6050 adivista l,12,5,performance,489200,0
489201,2017-01-17,0,0,0,幅の広めのラージサイズ。長時間の着用でも疲れを感じさせない人間工学から考えられたテンプル形状...,1,1,26,サングラス a164 6050 adivista l,21,5,performance,489201,0
489202,2017-01-25,0,0,0,幅の広めのラージサイズ。長時間の着用でも疲れを感じさせない人間工学から考えられたテンプル形状...,1,1,26,サングラス a164 6050 adivista l,12,5,performance,489202,0
489203,2017-01-30,0,0,0,幅の広めのラージサイズ。長時間の着用でも疲れを感じさせない人間工学から考えられたテンプル形状...,1,1,26,サングラス a164 6050 adivista l,12,5,performance,489203,0


In [25]:
# xtrain, xvalid, ytrain, yvalid = train_test_split(data_req[X], y, 
#                                                   stratify=y, 
#                                                   random_state=42, 
#                                                   test_size=0.1, shuffle=True)

In [26]:
train = data_req[data_req.date < "2018-05-01"]
valid = data_req[(data_req.date >= "2018-05-01") & (data_req.date <= "2018-07-31")]
test = data_req[data_req.date >= "2018-08-01"]

In [27]:
feather.write_dataframe(train, "train_japan_input_output")
feather.write_dataframe(valid, "valid_japan_input_output")
feather.write_dataframe(test, "test_japan_input_output")

In [28]:
max(train.date), min(valid.date), max(valid.date), min(test.date)

('2018-04-23', '2018-05-03', '2018-07-31', '2018-08-08')

In [29]:
len(y)

18

In [30]:
xtrain = train[X]
xvalid = valid[X]
xtest = test[X]

ytrain = train[y]
yvalid = valid[y]
ytest = test[y]

In [31]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿|¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s)

In [32]:
del data, data_req, train, test, valid

In [33]:
texts_train = [tokenize(x) for x in xtrain.description]
texts_valid = [tokenize(x) for x in xvalid.description]
texts_test = [tokenize(x) for x in xtest.description]

In [34]:
del xtrain, xvalid, xtest

In [35]:
segmenter = tinysegmenter.TinySegmenter()
tokenized_text_train = [segmenter.tokenize(x) for x in texts_train]
tokenized_text_valid = [segmenter.tokenize(x) for x in texts_valid]
tokenized_text_test = [segmenter.tokenize(x) for x in texts_test]

In [36]:
del texts_train, texts_valid, texts_test

In [39]:
# filename = "train_tokenizer_japan.pkl"
# filehandler = open(filename, 'wb')
# pickle.dump(tokenized_text_train, filehandler)

In [40]:
filename = "train_tokenizer_japan.pkl"
filehandler = open(filename, 'wb')
pickle.dump(tokenized_text_train, filehandler)

filename = "valid_tokenizer_japan.pkl"
filehandler = open(filename, 'wb')
pickle.dump(tokenized_text_valid, filehandler)

filename = "test_tokenizer_japan.pkl"
filehandler = open(filename, 'wb')
pickle.dump(tokenized_text_test, filehandler)

In [42]:
# file_pi1 = open('train_tokenizer_japan.pkl', 'rb')
# chk = pickle.load(file_pi1)

In [2]:
# file_pi1 = open('train_tokenizer_japan.pkl', 'rb')
# tokenized_text_train = pickle.load(file_pi1)

# file_pi2 = open('valid_tokenizer_japan.pkl', 'rb')
# tokenized_text_valid = pickle.load(file_pi2)

# file_pi3 = open('test_tokenizer_japan.pkl', 'rb')
# tokenized_text_test = pickle.load(file_pi3)

train = feather.read_dataframe("train_japan_input_output")
valid = feather.read_dataframe("valid_japan_input_output")
test = feather.read_dataframe("test_japan_input_output")

X = ["companyName", "distributorName", "ageGroup","description", "division", "gender", "group","name4", "sportsCategory", "subBrand"]
y = "SUBRAND_OUTPUT_ENC"

xtrain = train[X]
xvalid = valid[X]
xtest = test[X]

ytrain = train[y]
yvalid = valid[y]
ytest = test[y]

del train, valid, test

In [3]:
joined_token_train = [' '.join(x) for x in tokenized_text_train]
joined_token_valid = [' '.join(x) for x in tokenized_text_valid]
joined_token_test = [' '.join(x) for x in tokenized_text_test]

In [4]:
del tokenized_text_train,tokenized_text_valid, tokenized_text_test

In [5]:
filename = "joined_word_model.pkl"
filehandler = open(filename, 'wb')
pickle.dump(joined_token_train, filehandler)

filename = "joined_token_valid.pkl"
filehandler = open(filename, 'wb')
pickle.dump(joined_token_valid, filehandler)

filename = "joined_token_test.pkl"
filehandler = open(filename, 'wb')
pickle.dump(joined_token_test, filehandler)

In [24]:
file_pi1 = open('joined_token_train.pkl', 'rb')
joined_token_train = pickle.load(file_pi1)

file_pi2 = open('joined_token_valid.pkl', 'rb')
joined_token_valid = pickle.load(file_pi2)

file_pi3 = open('joined_token_test.pkl', 'rb')
joined_token_test = pickle.load(file_pi3)

In [25]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 300

token.fit_on_texts(joined_token_train + joined_token_valid + joined_token_test)
xtrain_seq = token.texts_to_sequences(joined_token_train)
xvalid_seq = token.texts_to_sequences(joined_token_valid)
xtest_seq = token.texts_to_sequences(joined_token_test)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [26]:
filename = "japan_xtrain_pad.pkl"
filehandler = open(filename, 'wb')
pickle.dump(xtrain_pad, filehandler)

filename = "japan_xvalid_pad.pkl"
filehandler = open(filename, 'wb')
pickle.dump(xvalid_pad, filehandler)

filename = "japan_xtest_pad.pkl"
filehandler = open(filename, 'wb')
pickle.dump(xtest_pad, filehandler)

filename = "japan_word_index.pkl"
filehandler = open(filename, 'wb')
pickle.dump(word_index, filehandler)

In [6]:
# Execute
file_pi1 = open('japan_word_index.pkl', 'rb')
word_index = pickle.load(file_pi1)

file_pi1 = open('japan_xtrain_pad.pkl', 'rb')
xtrain_pad = pickle.load(file_pi1)

file_pi1 = open('japan_xvalid_pad.pkl', 'rb')
xvalid_pad = pickle.load(file_pi1)

file_pi1 = open('japan_xtest_pad.pkl', 'rb')
xtest_pad = pickle.load(file_pi1)

In [7]:
token_keras_words = list(token.word_index)

In [10]:
from keras.preprocessing.text import text_to_word_sequence
keras_tokenised_words = [text_to_word_sequence(x, lower=False) for x in (joined_token_train + joined_token_valid + joined_token_test)]

In [27]:
del joined_token_train, joined_token_test, joined_token_valid

In [13]:
word_model = gensim.models.Word2Vec(keras_tokenised_words, size=300, min_count=1, window=5, iter=10)

In [14]:
del keras_tokenised_words

In [15]:
filename = "joined_word_model.pkl"
filehandler = open(filename, 'wb')
pickle.dump(word_model, filehandler)

In [28]:
# Execute
file_pi1 = open('joined_word_model.pkl', 'rb')
word_model = pickle.load(file_pi1)

## Execute from here

In [2]:
##### Everything together here. MY REFERENCE

train = feather.read_dataframe("train_japan_input_output")
valid = feather.read_dataframe("valid_japan_input_output")
test = feather.read_dataframe("test_japan_input_output")

X = ["companyName", "distributorName", "ageGroup","description", "division", "gender", "group","name4", "sportsCategory", "subBrand"]
y = "SUBRAND_OUTPUT_ENC"

xtrain = train[X]
xvalid = valid[X]
xtest = test[X]

ytrain = train[y]
yvalid = valid[y]
ytest = test[y]

del train, valid, test


file_pi1 = open('japan_word_index.pkl', 'rb')
word_index = pickle.load(file_pi1)

file_pi1 = open('japan_xtrain_pad.pkl', 'rb')
xtrain_pad = pickle.load(file_pi1)

file_pi1 = open('japan_xvalid_pad.pkl', 'rb')
xvalid_pad = pickle.load(file_pi1)

file_pi1 = open('japan_xtest_pad.pkl', 'rb')
xtest_pad = pickle.load(file_pi1)

file_pi1 = open('joined_word_model.pkl', 'rb')
word_model = pickle.load(file_pi1)

In [3]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = word_model.wv[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|████████████████████████████████████████████████████████████████████████| 35613/35613 [00:00<00:00, 145550.89it/s]


In [4]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-2.01717353, -0.99233973, -1.08852887, ..., -3.06032372,
         1.49680293, -1.42993021],
       [-0.50516093, -0.59006751, -1.52408195, ..., -1.79377186,
         0.24416398, -3.26893067],
       ...,
       [ 0.04895103, -0.03756456, -0.01938329, ..., -0.00814213,
        -0.04584658, -0.08762853],
       [ 0.00822348, -0.0179678 ,  0.033421  , ..., -0.03149183,
        -0.02039536,  0.0701666 ],
       [ 0.05051522,  0.02162825, -0.04719217, ..., -0.00573608,
         0.04827156, -0.06946177]])

In [5]:
# Input: meant to receive sequences of 300 integers
main_input = Input(shape=(300,), name='main_input')
max_len = 300
# This embedding layer will encode the input sequence
# into a sequence of dense 300-dimensional vectors.
x = Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False)(main_input)

x = SpatialDropout1D(0.3)(x)


# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = LSTM(300, dropout=0.3, recurrent_dropout=0.3)(x)

In [6]:
auxiliary_input_1 = Input((1,), name='aux_input_1')
auxiliary_input_2 = Input((1,), name='aux_input_2')
auxiliary_input_3 = Input((1,), name='aux_input_3')
auxiliary_input_4 = Input((1,), name='aux_input_4')
auxiliary_input_5 = Input((1,), name='aux_input_5')
auxiliary_input_6 = Input((1,), name='aux_input_6')
auxiliary_input_7 = Input((1,), name='aux_input_7')
auxiliary_input_8 = Input((1,), name='aux_input_8')
auxiliary_input_9 = Input((1,), name='aux_input_9')

x = keras.layers.concatenate([lstm_out, auxiliary_input_1, auxiliary_input_2, auxiliary_input_3, auxiliary_input_4, 
                             auxiliary_input_5, auxiliary_input_6, auxiliary_input_7, 
                             auxiliary_input_8, auxiliary_input_9], axis = 1)

# We stack a deep densely-connected network on top
x = Dense(1024, activation='relu')(x)
x = Dropout(0.7)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

# Output softmax layer
main_output = Dense(2, activation='softmax', name='main_output')(x)

In [7]:
model = Model(inputs=[main_input, auxiliary_input_1, auxiliary_input_2, auxiliary_input_3, auxiliary_input_4, auxiliary_input_5, auxiliary_input_6, auxiliary_input_7, auxiliary_input_8, auxiliary_input_9], outputs=main_output)
rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0, clipvalue=0.5)
model.compile(loss='categorical_crossentropy', optimizer= 'adam')

In [8]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)
ytest_enc = np_utils.to_categorical(ytest)

In [9]:
# le = preprocessing.LabelEncoder()
# xtrain_company = np.array(xtrain.COMPANY)
# xtrain_company = le.fit_transform(xtrain_company)

In [10]:
# xvalid_company = np.array(xvalid.COMPANY)
# xvalid_company = le.fit_transform(xvalid_company)

In [9]:
le = preprocessing.LabelEncoder()

xtrain_companyName = np.array(xtrain.companyName)
xtrain_companyName = le.fit_transform(xtrain_companyName)
# xtrain_companyName = (xtrain_companyName - xtrain_companyName.mean())/xtrain_companyName.std()

xtrain_distributorName = np.array(xtrain.distributorName)
xtrain_distributorName = le.fit_transform(xtrain_distributorName)
# xtrain_distributorName = (xtrain_distributorName - xtrain_distributorName.mean())/xtrain_distributorName.std()

xtrain_ageGroup = np.array(xtrain.ageGroup)
xtrain_ageGroup = le.fit_transform(xtrain_ageGroup)
# xtrain_ageGroup = (xtrain_ageGroup - xtrain_ageGroup.mean())/xtrain_ageGroup.std()

xtrain_division = np.array(xtrain.division)
xtrain_division = le.fit_transform(xtrain_division)
# xtrain_division = (xtrain_division - xtrain_division.mean())/xtrain_division.std()

xtrain_gender = np.array(xtrain.gender)
xtrain_gender = le.fit_transform(xtrain_gender)
# xtrain_gender = (xtrain_gender - xtrain_gender.mean())/xtrain_gender.std()

xtrain_group = np.array(xtrain.group)
xtrain_group = le.fit_transform(xtrain_group)
# xtrain_group = (xtrain_group - xtrain_group.mean())/xtrain_group.std()

xtrain_name4 = np.array(xtrain.name4)
xtrain_name4 = le.fit_transform(xtrain_name4)
# xtrain_name4 = (xtrain_name4 - xtrain_name4.mean())/xtrain_name4.std()


xtrain_sportsCategory = np.array(xtrain.sportsCategory)
xtrain_sportsCategory = le.fit_transform(xtrain_sportsCategory)
# xtrain_sportsCategory = (xtrain_sportsCategory - xtrain_sportsCategory.mean())/xtrain_sportsCategory.std()


xtrain_subBrand = np.array(xtrain.subBrand)
xtrain_subBrand = le.fit_transform(xtrain_subBrand)
# xtrain_subBrand = (xtrain_subBrand - xtrain_subBrand.mean())/xtrain_subBrand.std()

In [12]:
#xtrain_sportsCategory

In [10]:
xvalid_companyName = np.array(xvalid.companyName)
xvalid_companyName = le.fit_transform(xvalid_companyName)
# xvalid_companyName = (xvalid_companyName - xvalid_companyName.mean())/xvalid_companyName.std()

xvalid_distributorName = np.array(xvalid.distributorName)
xvalid_distributorName = le.fit_transform(xvalid_distributorName)
# xvalid_distributorName = (xvalid_distributorName - xvalid_distributorName.mean())/xvalid_distributorName.std()

xvalid_ageGroup = np.array(xvalid.ageGroup)
xvalid_ageGroup = le.fit_transform(xvalid_ageGroup)
# xvalid_ageGroup = (xvalid_ageGroup - xvalid_ageGroup.mean())/xvalid_ageGroup.std()

xvalid_division = np.array(xvalid.division)
xvalid_division = le.fit_transform(xvalid_division)
# xvalid_division = (xvalid_division - xvalid_division.mean())/xvalid_division.std()

xvalid_gender = np.array(xvalid.gender)
xvalid_gender = le.fit_transform(xvalid_gender)
# xvalid_gender = (xvalid_gender - xvalid_gender.mean())/xvalid_gender.std()

xvalid_group = np.array(xvalid.group)
xvalid_group = le.fit_transform(xvalid_group)
# xvalid_group = (xvalid_group - xvalid_group.mean())/xvalid_group.std()

xvalid_name4 = np.array(xvalid.name4)
xvalid_name4 = le.fit_transform(xvalid_name4)
# xvalid_name4 = (xvalid_name4 - xvalid_name4.mean())/xvalid_name4.std()

xvalid_sportsCategory = np.array(xvalid.sportsCategory)
xvalid_sportsCategory = le.fit_transform(xvalid_sportsCategory)
# xvalid_sportsCategory = (xvalid_sportsCategory - xvalid_sportsCategory.mean())/xvalid_sportsCategory.std()

xvalid_subBrand = np.array(xvalid.subBrand)
xvalid_subBrand = le.fit_transform(xvalid_subBrand)
# xvalid_subBrand = (xvalid_subBrand - xvalid_subBrand.mean())/xvalid_subBrand.std()

In [14]:
xtest_companyName = np.array(xtest.companyName)
xtest_companyName = le.fit_transform(xtest_companyName)
# xtest_companyName = (xtest_companyName - xtest_companyName.mean())/xtest_companyName.std()

xtest_distributorName = np.array(xtest.distributorName)
xtest_distributorName = le.fit_transform(xtest_distributorName)
# xtest_distributorName = (xtest_distributorName - xtest_distributorName.mean())/xtest_distributorName.std()

xtest_ageGroup = np.array(xtest.ageGroup)
xtest_ageGroup = le.fit_transform(xtest_ageGroup)
# xtest_ageGroup = (xtest_ageGroup - xtest_ageGroup.mean())/xtest_ageGroup.std()

xtest_division = np.array(xtest.division)
xtest_division = le.fit_transform(xtest_division)
# xtest_division = (xtest_division - xtest_division.mean())/xtest_division.std()

xtest_gender = np.array(xtest.gender)
xtest_gender = le.fit_transform(xtest_gender)
# xtest_gender = (xtest_gender - xtest_gender.mean())/xtest_gender.std()

xtest_group = np.array(xtest.group)
xtest_group = le.fit_transform(xtest_group)
# xtest_group = (xtest_group - xtest_group.mean())/xtest_group.std()


xtest_name4 = np.array(xtest.name4)
xtest_name4 = le.fit_transform(xtest_name4)
# xtest_name4 = (xtest_name4 - xtest_name4.mean())/xtest_name4.std()

xtest_sportsCategory = np.array(xtest.sportsCategory)
xtest_sportsCategory = le.fit_transform(xtest_sportsCategory)
# xtest_sportsCategory = (xtest_sportsCategory - xtest_sportsCategory.mean())/xtest_sportsCategory.std()

xtest_subBrand = np.array(xtest.subBrand)
xtest_subBrand = le.fit_transform(xtest_subBrand)
# xtest_subBrand = (xtest_subBrand - xtest_subBrand.mean())/xtest_subBrand.std()

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 300)     10684200    main_input[0][0]                 
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 300, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 300)          721200      spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
aux_input_

In [ ]:
model.fit(x = [xtrain_pad, xtrain_companyName, xtrain_distributorName, xtrain_ageGroup, \
               xtrain_division, xtrain_gender, xtrain_group, xtrain_name4, xtrain_sportsCategory, \
               xtrain_subBrand] , y=ytrain_enc, batch_size=128, epochs=2, verbose=1, 
               validation_data=([xvalid_pad, xvalid_companyName, xvalid_distributorName, xvalid_ageGroup, \
                                 xvalid_division, xvalid_gender, xvalid_group, xvalid_name4, xvalid_sportsCategory, \
                                 xvalid_subBrand], yvalid_enc))

Train on 1125047 samples, validate on 222889 samples
Epoch 1/2


  34560/1125047 [..............................] - ETA: 18:32:35 - loss: 9.722 - ETA: 15:38:28 - loss: 8.707 - ETA: 14:38:05 - loss: 8.113 - ETA: 14:03:27 - loss: 7.916 - ETA: 13:43:20 - loss: 7.966 - ETA: 13:27:14 - loss: 7.835 - ETA: 13:14:39 - loss: 7.903 - ETA: 13:09:33 - loss: 7.616 - ETA: 13:07:10 - loss: 7.441 - ETA: 13:05:53 - loss: 7.365 - ETA: 13:01:48 - loss: 7.313 - ETA: 13:01:45 - loss: 7.256 - ETA: 13:00:19 - loss: 7.260 - ETA: 12:57:28 - loss: 7.200 - ETA: 12:58:59 - loss: 7.232 - ETA: 12:59:35 - loss: 7.181 - ETA: 12:59:54 - loss: 7.122 - ETA: 13:00:28 - loss: 7.069 - ETA: 13:00:28 - loss: 7.081 - ETA: 13:00:17 - loss: 7.111 - ETA: 12:59:20 - loss: 7.042 - ETA: 12:58:32 - loss: 7.043 - ETA: 12:58:49 - loss: 7.026 - ETA: 12:59:10 - loss: 7.090 - ETA: 12:57:52 - loss: 7.089 - ETA: 12:58:50 - loss: 7.034 - ETA: 12:59:11 - loss: 7.007 - ETA: 12:57:55 - loss: 7.058 - ETA: 12:58:12 - loss: 7.079 - ETA: 12:58:52 - loss: 7.057 - ETA: 13:03:10 - loss: 7.045 - ETA: 13:05:29 - los

 104192/1125047 [=>............................] - ETA: 31:08:45 - loss: 6.974 - ETA: 31:10:48 - loss: 6.975 - ETA: 31:12:22 - loss: 6.975 - ETA: 31:13:50 - loss: 6.975 - ETA: 31:15:08 - loss: 6.976 - ETA: 31:16:43 - loss: 6.974 - ETA: 31:18:30 - loss: 6.972 - ETA: 31:19:52 - loss: 6.973 - ETA: 31:21:27 - loss: 6.972 - ETA: 31:23:11 - loss: 6.971 - ETA: 31:24:21 - loss: 6.971 - ETA: 31:25:34 - loss: 6.971 - ETA: 31:27:08 - loss: 6.971 - ETA: 31:27:34 - loss: 6.972 - ETA: 31:27:45 - loss: 6.973 - ETA: 31:27:59 - loss: 6.975 - ETA: 31:28:28 - loss: 6.977 - ETA: 31:28:36 - loss: 6.978 - ETA: 31:28:32 - loss: 6.978 - ETA: 31:28:48 - loss: 6.978 - ETA: 31:29:15 - loss: 6.976 - ETA: 31:29:26 - loss: 6.976 - ETA: 31:29:41 - loss: 6.975 - ETA: 31:30:08 - loss: 6.975 - ETA: 31:30:36 - loss: 6.975 - ETA: 31:30:49 - loss: 6.976 - ETA: 31:31:08 - loss: 6.974 - ETA: 31:31:34 - loss: 6.974 - ETA: 31:31:49 - loss: 6.973 - ETA: 31:31:58 - loss: 6.971 - ETA: 31:32:22 - loss: 6.970 - ETA: 31:32:48 - los

 139008/1125047 [==>...........................] - ETA: 28:42:23 - loss: 6.970 - ETA: 28:41:05 - loss: 6.969 - ETA: 28:39:46 - loss: 6.969 - ETA: 28:38:25 - loss: 6.970 - ETA: 28:37:06 - loss: 6.969 - ETA: 28:35:49 - loss: 6.969 - ETA: 28:34:28 - loss: 6.969 - ETA: 28:33:07 - loss: 6.968 - ETA: 28:31:50 - loss: 6.968 - ETA: 28:30:32 - loss: 6.968 - ETA: 28:29:13 - loss: 6.968 - ETA: 28:27:55 - loss: 6.967 - ETA: 28:26:41 - loss: 6.968 - ETA: 28:25:22 - loss: 6.969 - ETA: 28:24:04 - loss: 6.970 - ETA: 28:22:50 - loss: 6.971 - ETA: 28:21:37 - loss: 6.971 - ETA: 28:20:21 - loss: 6.971 - ETA: 28:19:04 - loss: 6.971 - ETA: 28:17:48 - loss: 6.971 - ETA: 28:16:33 - loss: 6.973 - ETA: 28:15:16 - loss: 6.973 - ETA: 28:14:01 - loss: 6.975 - ETA: 28:12:46 - loss: 6.975 - ETA: 28:11:30 - loss: 6.974 - ETA: 28:10:13 - loss: 6.974 - ETA: 28:08:59 - loss: 6.975 - ETA: 28:07:43 - loss: 6.975 - ETA: 28:06:26 - loss: 6.974 - ETA: 28:05:11 - loss: 6.973 - ETA: 28:03:57 - loss: 6.972 - ETA: 28:02:42 - los

 173824/1125047 [===>..........................] - ETA: 24:07:16 - loss: 6.971 - ETA: 24:06:30 - loss: 6.970 - ETA: 24:05:42 - loss: 6.971 - ETA: 24:04:53 - loss: 6.971 - ETA: 24:04:07 - loss: 6.971 - ETA: 24:03:22 - loss: 6.971 - ETA: 24:02:33 - loss: 6.971 - ETA: 24:01:45 - loss: 6.971 - ETA: 24:00:59 - loss: 6.972 - ETA: 24:00:12 - loss: 6.972 - ETA: 23:59:23 - loss: 6.973 - ETA: 23:58:36 - loss: 6.974 - ETA: 23:57:50 - loss: 6.973 - ETA: 23:57:02 - loss: 6.973 - ETA: 23:56:15 - loss: 6.972 - ETA: 23:55:30 - loss: 6.972 - ETA: 23:54:44 - loss: 6.972 - ETA: 23:53:56 - loss: 6.972 - ETA: 23:53:09 - loss: 6.973 - ETA: 23:52:24 - loss: 6.972 - ETA: 23:51:37 - loss: 6.971 - ETA: 23:50:49 - loss: 6.971 - ETA: 23:50:04 - loss: 6.971 - ETA: 23:49:18 - loss: 6.970 - ETA: 23:48:31 - loss: 6.971 - ETA: 23:47:46 - loss: 6.971 - ETA: 23:47:00 - loss: 6.972 - ETA: 23:46:13 - loss: 6.972 - ETA: 23:45:26 - loss: 6.973 - ETA: 23:44:41 - loss: 6.973 - ETA: 23:43:56 - loss: 6.972 - ETA: 23:43:09 - los

 208640/1125047 [====>.........................] - ETA: 21:09:19 - loss: 6.967 - ETA: 21:08:46 - loss: 6.967 - ETA: 21:08:12 - loss: 6.968 - ETA: 21:07:39 - loss: 6.969 - ETA: 21:07:08 - loss: 6.969 - ETA: 21:06:35 - loss: 6.969 - ETA: 21:06:02 - loss: 6.969 - ETA: 21:05:29 - loss: 6.970 - ETA: 21:04:56 - loss: 6.970 - ETA: 21:04:23 - loss: 6.970 - ETA: 21:03:53 - loss: 6.971 - ETA: 21:03:21 - loss: 6.970 - ETA: 21:02:49 - loss: 6.970 - ETA: 21:02:16 - loss: 6.970 - ETA: 21:01:43 - loss: 6.970 - ETA: 21:01:12 - loss: 6.970 - ETA: 21:00:38 - loss: 6.970 - ETA: 21:00:08 - loss: 6.970 - ETA: 20:59:45 - loss: 6.970 - ETA: 20:59:22 - loss: 6.969 - ETA: 20:58:50 - loss: 6.970 - ETA: 20:58:17 - loss: 6.970 - ETA: 20:57:46 - loss: 6.970 - ETA: 20:57:13 - loss: 6.970 - ETA: 20:56:41 - loss: 6.970 - ETA: 20:56:09 - loss: 6.970 - ETA: 20:55:38 - loss: 6.969 - ETA: 20:55:05 - loss: 6.969 - ETA: 20:54:33 - loss: 6.970 - ETA: 20:54:03 - loss: 6.970 - ETA: 20:53:32 - loss: 6.970 - ETA: 20:52:59 - los

 231168/1125047 [=====>........................] - ETA: 19:01:28 - loss: 6.972 - ETA: 19:01:04 - loss: 6.972 - ETA: 19:00:39 - loss: 6.972 - ETA: 19:00:14 - loss: 6.972 - ETA: 18:59:51 - loss: 6.972 - ETA: 18:59:26 - loss: 6.972 - ETA: 18:59:00 - loss: 6.972 - ETA: 18:58:37 - loss: 6.972 - ETA: 18:58:12 - loss: 6.971 - ETA: 18:57:47 - loss: 6.972 - ETA: 18:57:23 - loss: 6.972 - ETA: 18:56:59 - loss: 6.972 - ETA: 18:56:33 - loss: 6.972 - ETA: 18:56:08 - loss: 6.972 - ETA: 18:55:45 - loss: 6.972 - ETA: 18:55:21 - loss: 6.972 - ETA: 18:54:56 - loss: 6.971 - ETA: 18:54:32 - loss: 6.970 - ETA: 18:54:08 - loss: 6.970 - ETA: 18:53:43 - loss: 6.971 - ETA: 18:53:17 - loss: 6.971 - ETA: 18:52:53 - loss: 6.971 - ETA: 18:52:29 - loss: 6.971 - ETA: 18:52:04 - loss: 6.971 - ETA: 18:51:40 - loss: 6.970 - ETA: 18:51:16 - loss: 6.970 - ETA: 18:50:52 - loss: 6.970 - ETA: 18:50:27 - loss: 6.971 - ETA: 18:50:03 - loss: 6.971 - ETA: 18:49:39 - loss: 6.970 - ETA: 18:49:16 - loss: 6.970 - ETA: 18:48:55 - los

In [ ]:
model.save("Keras_model_Adam_BS128_V1.h5")

In [ ]:
from keras.models import load_model
model = load_model('Keras_model_Adam_BS128_V1.h5')

In [30]:
xtrain_company.shape

(55381,)

In [33]:
predictions = model.predict([xtest_pad, xtest_companyName, xtest_distributorName, xtest_ageGroup, \
                             xtest_division, xtest_gender, xtest_group, xtest_name4, xtest_sportsCategory, \
                             xtest_subBrand])

In [34]:
predict_label = [np.argmax(i) for i in predictions]

In [35]:
predict_label = lbl_enc.inverse_transform(predict_label)

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
yvalid_label = lbl_enc.inverse_transform(yvalid)

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [37]:
test_df = pd.DataFrame({'description' : xvalid.description, 'actuals' : yvalid_label, 'predictions' : predict_label})

In [39]:
f1_score(test_df.actuals, test_df.predictions,average="macro")

0.8347948792989917

In [40]:
accuracy_score(test_df.actuals, test_df.predictions)

0.8378290542736432